# Homework

## Dataset

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'
!wget $data -O homework-3.csv

--2025-10-13 21:59:03--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘homework-3.csv’

homework-3.csv      100%[===================>]  78.98K   469KB/s    in 0.2s    

2025-10-13 21:59:04 (469 KB/s) - ‘homework-3.csv’ saved [80876/80876]



In [3]:
df = pd.read_csv('homework-3.csv')

In [4]:
len(df)

1462

## Data preparation

In [7]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [8]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

In [9]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [11]:
df[categorical] = df[categorical].fillna('NA')

In [13]:
df[numerical] = df[numerical].fillna(0.0)

## Question 1

What is the most frequent observation (mode) for the column industry?

In [19]:
df['industry'].mode()

0    retail
Name: industry, dtype: object

## Question 2

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

In [22]:
df[numerical].corr().abs()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,0.023565,0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,0.023565,0.027036,1.000000,0.009888
lead_score,0.004879,0.015610,0.009888,1.000000


In [23]:
from sklearn.model_selection import train_test_split

In [24]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [29]:
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

In [31]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [32]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [33]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

## Question 3

- Calculate the mutual information score between `converted` and other categorical variables in the dataset. Use the training set only.
- Round the scores to 2 decimals using `round(score, 2)`.

Which of these variables has the biggest mutual information score?

In [36]:
from sklearn.metrics import mutual_info_score

In [55]:
def mutual_info_convered_score(series):
    return mutual_info_score(series, y_train)

In [56]:
mi = df_train[categorical].apply(mutual_info_convered_score)
mi.sort_values(ascending=False)

lead_source          0.035396
employment_status    0.012938
industry             0.011575
location             0.004464
dtype: float64

## Question 4

- Now let's train a logistic regression.
- Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
- Fit the model on the training dataset.
  - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
  - `model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)`
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

In [48]:
from sklearn.feature_extraction import DictVectorizer

In [49]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [52]:
from sklearn.linear_model import LogisticRegression

In [53]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [54]:
y_pred = model.predict_proba(X_val)[:, 1]

In [57]:
converted_decision = (y_pred >= 0.5)

In [59]:
round((y_val == converted_decision).mean(), 2)

np.float64(0.7)

## Question 5

- Let's find the least useful feature using the feature elimination technique.
- Train a model using the same features and parameters as in Q4 (without rounding).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

In [67]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_val)[:, 1]
converted_decision = (y_pred >= 0.5)

original_accuracy = (y_val == converted_decision).mean()

In [73]:
eliminated_features = ['industry', 'employment_status', 'lead_score']
for eliminated_feature in eliminated_features:
    features = categorical + numerical
    dv = DictVectorizer(sparse=False)

    curr_df_train = df_train[features].drop(eliminated_feature, axis=1)
    curr_train_dict = curr_df_train.to_dict(orient='records')
    X_curr_train = dv.fit_transform(curr_train_dict)

    curr_df_val = df_val[features].drop(eliminated_feature, axis=1)
    curr_val_dict = curr_df_val.to_dict(orient='records')
    X_curr_val = dv.transform(curr_val_dict)
    
    curr_model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    curr_model.fit(X_curr_train, y_train)
    
    y_curr_pred = curr_model.predict_proba(X_curr_val)[:, 1]
    converted_decision = (y_curr_pred >= 0.5)
    
    curr_accuracy = (y_val == converted_decision).mean()
    print('eliminated_feature: ', eliminated_feature, ' original accuracy: ', original_accuracy, ' current accuracy: ', curr_accuracy,  ' accuracy difference: ', abs(original_accuracy - curr_accuracy))
    

eliminated_feature:  industry  original accuracy:  0.6996587030716723  current accuracy:  0.6996587030716723  accuracy difference:  0.0
eliminated_feature:  employment_status  original accuracy:  0.6996587030716723  current accuracy:  0.6962457337883959  accuracy difference:  0.0034129692832763903
eliminated_feature:  lead_score  original accuracy:  0.6996587030716723  current accuracy:  0.7064846416382252  accuracy difference:  0.0068259385665528916


## Question 6

- Now let's train a regularized logistic regression.
- Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
- Train models using all the features as in Q4.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these C leads to the best accuracy on the validation set?

In [74]:
C_values = [0.01, 0.1, 1, 10, 100]

In [88]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

for C_value in C_values:
    model = LogisticRegression(solver='liblinear', C=C_value, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict_proba(X_val)[:, 1]
    converted_decision = (y_pred >= 0.5)
    
    accuracy = round((y_val == converted_decision).mean(), 3)
    print('C value: ', C_value, ' accuracy: ', accuracy)

C value:  0.01  accuracy:  0.7
C value:  0.1  accuracy:  0.7
C value:  1  accuracy:  0.7
C value:  10  accuracy:  0.7
C value:  100  accuracy:  0.7
